In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from LPRnet.LPRnet_separable import *
from LPRnet.LPRnet_edgeTPU import LPRnet as edgetpu
from generator import DataGenerator, RealDataGenerator
import time
PROJECT_NAME = "LPRnet_keras"
MODEL_NAME = "ccpd002"

IMAGE_SHAPE = [94,24]
CHARS = "ABCDEFGHIJKLMNPQRSTUVWXYZ0123456789" # exclude I, O
CHARS_DICT = {char:i for i, char in enumerate(CHARS)}
DECODE_DICT = {i:char for i, char in enumerate(CHARS)}
NUM_CLASS = len(CHARS)+1

In [ ]:
MODEL_PATH = 'trained_models'
TFLITE_PATH = 'tflite_models'
model = keras.models.load_model(
    os.path.join(MODEL_PATH, MODEL_NAME), custom_objects={"global_context": global_context, "CTCLoss": CTCLoss  }
)
model.summary()

In [ ]:
import glob
import cv2

real_images_val = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\valid\\*\\*.png')
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\marty\\*\\*.png')

data = []
labels = []
val_data = []
val_labels = []

for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    label = label.replace("O","0")
    image = cv2.imread(file,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(94,24))/256
    data.append(image)
    labels.append([CHARS_DICT[i] for i in label.split('_')[0]])

for file in real_images_val:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    label = label.replace("O","0")
    image = cv2.imread(file,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(94,24))
    val_data.append(image)
    val_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])

training_set = np.array(data,dtype=np.float32)
training_labels = np.array(labels)
ragged = tf.ragged.constant(training_labels).to_tensor()
real_dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).batch(64)


In [ ]:
if os.path.exists("trained_models//{}".format(MODEL_NAME)):
    model = keras.models.load_model(
        "trained_models//{}".format(MODEL_NAME), custom_objects={"global_context": global_context, "CTCLoss": CTCLoss  }
    )
    print("loading model")
else:
    model = LPRnet()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),loss =CTCLoss)
    model.build((1,24,94,3))
    print("creating model")

In [ ]:
model = edgetpu()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),loss =CTCLoss)
model.build((1,24,94,3))


In [23]:
model.save("trained_models//ccpd002.h5")
m = keras.models.load_model("trained_models//ccpd002.h5",custom_objects={ "CTCLoss": CTCLoss  })

In [24]:
m.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 24, 94, 3)]  0           []                               
                                                                                                  
 sequential_4 (Sequential)      (None, 6, 23, 10)    3194        ['input_3[0][0]']                
                                                                                                  
 tf.reshape_14 (TFOpLambda)     (None, 1380)         0           ['sequential_4[0][0]']           
                                                                                                  
 sequential_5 (Sequential)      (None, 6)            44390       ['tf.reshape_14[0][0]']          
                                                                                            

In [ ]:
from gen_plates_keras import *
gen = ImageGenerator()
def generate_dataset(num = 100):
    data, labels = gen.generate_images(num)
    gen_labels = []
    for label in labels:
        gen_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])
    pics =np.array(data)
    labels = np.array(labels)
    training_set = np.array(pics,dtype=np.float32)
    training_labels = np.array(gen_labels)
    ragged = tf.ragged.constant(training_labels).to_tensor()
    dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).shuffle(640).batch(64)
    return dataset
test_dataset = generate_dataset()

In [ ]:

TFLITE_PATH = 'tflite_models'
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
]
tflite_model = converter.convert()


with open("./{}/{}.tflite".format(TFLITE_PATH,MODEL_NAME), 'wb') as f:
  f.write(tflite_model)

# Testing Keras Model Accuracy

In [ ]:
import glob
import cv2
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\datasets\\custom\\CCPD-PLATES-TEST\\*.png')
ctr = 0
batch = []
for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    label = label.replace('O',str(0))
    image = cv2.imread(file)
    test_image = image.astype(np.float32)
    test_image = cv2.resize(image,(94,24))/256
    test_image = np.expand_dims(test_image,axis=0)
    preds = model(test_image) 
    print(preds.shape)
    #decoded = keras.backend.ctc_decode(np.expand_dims(preds,0),(24,))
   
    decoded = keras.backend.ctc_decode(preds,(24,))
    text = ""
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            text += DECODE_DICT[i]
    if label == text:
        ctr += 1
    else: 
        print(text, label)
print(ctr,len(real_images))

# Testing Tflite model accuracy


In [ ]:
interpreter = tf.lite.Interpreter("C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\tflite_models\\{}.tflite".format(MODEL_NAME))
#interpreter = tf.lite.Interpreter("C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\tflite_models\\keras_lprnet_separable.tflite")
import numpy as np
import time 
import glob
import cv2
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.resize_tensor_input(input_details[0]['index'], (1, 24, 94, 3))
interpreter.allocate_tensors()
#real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\data\\marty\\*\\*.png')
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\datasets\\custom\\CCPD-PLATES-TEST\\*.png')
start = time.time()
ctr = 0
batch_images=[]
batch_labels = []
for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    batch_labels.append(label)
    image = cv2.imread(file)
    test_image = cv2.resize(image,(94,24))/256
    test_image = test_image.astype(np.float32)
    batch_images.append(test_image)
    test_image = np.expand_dims(test_image,axis=0)
    interpreter.set_tensor(input_details[0]['index'], test_image)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])

    #decoded = keras.backend.ctc_decode(output_data,(24,))
    decoded = keras.backend.ctc_decode(np.expand_dims(preds,0),(24,))
    text = ""
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            text += DECODE_DICT[i]
    if label == text:
        ctr += 1
    else:
        print(text, " "+ label)
print(time.time()-start)
print(ctr,len(real_images))

In [ ]:
tflite_interpreter = tf.lite.Interpreter("C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\tflite_models\\{}.tflite".format(MODEL_NAME))
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()
tflite_interpreter.resize_tensor_input(input_details[0]['index'], (193, 24, 94, 3))
tflite_interpreter.resize_tensor_input(output_details[0]['index'], (193, 24,36))
tflite_interpreter.allocate_tensors()

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()
print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])
start = time.time()
batch = np.array(batch_images)
tflite_interpreter.set_tensor(input_details[0]['index'], batch)
tflite_interpreter.invoke()
output_data = tflite_interpreter.get_tensor(output_details[0]['index'])
ctr = 0
for k,v in enumerate(output_data):
        text = ""
        print(v.shape)
        undecoded=np.expand_dims(v,axis=0)
        decoded = keras.backend.ctc_decode(undecoded,(24,),greedy=True)
        for j in np.array(decoded[0]).reshape(24):
            if j >-1:
                text += DECODE_DICT[j]
        if batch_labels[k] == text:
            print(text)
            ctr += 1
print(ctr)
print(time.time()-start)
print(ctr,len(real_images))

In [ ]:
import glob
import cv2
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\valid\\*\\*.png')
ctr = 0
batch = []
for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    label = label.replace('O',str(0))
    image = cv2.imread(file).astype('float32')
    test_image = cv2.resize(image,(94,24))/256
    batch.append(test_image)
    test_image = np.expand_dims(test_image,axis=0)
    preds = model.predict(test_image) 
    decoded = tf.keras.backend.ctc_decode(preds,(24,))
    text = ""
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            text += DECODE_DICT[i]
    if label == text or label == text[:-1] or label == text[1:]:
        ctr += 1
    else:
        print(text," "+ label)
print(ctr)
print(len(real_images))

In [ ]:
val_training_set = np.array(val_data,dtype=np.float32)
val_training_labels = np.array(val_labels)
val_ragged = tf.ragged.constant(val_training_labels).to_tensor()
val_dataset = tf.data.Dataset.from_tensor_slices((val_training_set,val_ragged)).shuffle(640).batch(64)

In [ ]:
import cv2
from gen_plates_keras import ImageGenerator, RealImageGenerator

gen = RealImageGenerator()

In [ ]:
images,labels = gen.generate_images(100,training=False)
for im,label in zip(images,labels):
    cv2.imshow("window",im)
    print(label)
    cv2.waitKey(0)
cv2.destroyAllWindows()